# Multiple simulations of the MARM1 model

This notebook runs multiple sequential simulations of the MARM1 model in order to generate a dataset for multiple conditions. For a detailed description of how a single simulation run is set up and the visulization of time-course trajectories please refer to the companion notebook *MARM1_simulation_single_run.ipynb*. 

Note: Execution of all ~9000 simulations will take on the order of a full day on a single CPU. (In practice we used a variant of this code that splits the work across many CPUs on an HPC cluster)  This notebook will only run the first 4 simulations by default, but you can run them all by changing the following variable to `True`. 

In [1]:
do_all_simulations = False

Import the model and libraries necessary to run MARM1 model simulations. 

In [2]:
%matplotlib notebook
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
#load model 
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern
from MARM1 import model

Generating the condition settings (e.g. RAFi and MEKi concentrations) for each model simulation to be run.

In [3]:
n_doses = 11
lb_RAFi = -4
ub_RAFi = 1
lb_MEKi = -5
ub_MEKi = 0
RAFi_concentration = np.append([0], np.logspace(lb_RAFi, ub_RAFi, n_doses))
MEKi_concentration = np.append([0], np.logspace(lb_MEKi, ub_MEKi, n_doses))
t_pretrt_fixed = [24]
t_pretrt = [0.83, 0.25, 0.5, 1, 2, 4, 8]
EGF_concentration = [0, 100]
t_trt = [2]
param_set_index = range(100)

settings_list = sorted(itertools.chain(
    # ndoses x ndoses matrix of RAFi and MEKi dose combinations with 24h drug treatment followed by 2h of EGF stimulation
    itertools.product([0], t_pretrt_fixed, RAFi_concentration, MEKi_concentration, t_trt, EGF_concentration),
    # RAFi dose-response 
    itertools.product(param_set_index, t_pretrt_fixed, RAFi_concentration, [0.0], t_trt, EGF_concentration),
    # MEKi dose-response 
    itertools.product(param_set_index, t_pretrt_fixed, [0.0], MEKi_concentration, t_trt, EGF_concentration),
    # fixed RAFi + MEKi dose-response
    itertools.product(param_set_index, t_pretrt_fixed, [1.0], MEKi_concentration, t_trt, EGF_concentration),
    # different RAFi treatment durations
    itertools.product(param_set_index, t_pretrt, [1.0], [0.0], t_trt, EGF_concentration),
))

In [4]:
print(len(settings_list), "total simulations")

8888 total simulations


Define support functions to run simulations. 

In [5]:
def equilibrate(simulator, initials):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 1e-4
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        #print(f"    at t={tspan[-1]:<5.3g} ... ", end='', flush=True)
        res = simulator.run(tspan=tspan, initials=initials)
        df = pd.concat([df, res.dataframe.iloc[1:]])
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(simulator.model.species)
        #print(f"{cs}/{n} species converged")
        if np.all(close):
            break
        tspan *= scale
    return df

In [6]:
def get_species_index(model, pattern):
    """Return the integer species number for a given species in the model"""
    pattern = as_complex_pattern(pattern)
    matches = [
        i for i, s in enumerate(model.species)
        if s.is_equivalent_to(pattern)
    ]
    n = len(matches)
    assert n == 1, f"Expected exactly one match, got {n}"
    return matches[0]

## Simulate the model over multiple conditions

In [7]:
if not do_all_simulations:
    settings_list = settings_list[:4]
    print("Only running first 4 simulations")

N_time_points = 97
out_filename = 'trajectories_multiple_runs.csv'

Only running first 4 simulations


Here we loop over the previously generated list of conditions and run a simulation for each one, saving all the resulting trajectories to a single CSV file.

In [8]:
param_prev = -1
for i, settings in enumerate(tqdm_notebook(settings_list, desc='Simulation progress')):
   
    [param, pretrt, rafi, meki, trt, egfc] = settings
    
    # run a simulation of the unperturbed model to obtain initial steady state (if not run before)
    if param != param_prev:
        param_sets = pd.read_csv('parameter_sets.csv', index_col=0)
        param_sets = param_sets.drop('chi2', axis=1)
        params = param_sets.iloc[param].to_dict()
        sim = ScipyOdeSimulator(model, param_values=params, atol=1E-50)
        df_eq = equilibrate(sim, None)

    # run a time-course simulation for the pretreatment phase
    RAFi_index = get_species_index(model, model.monomers.RAFi(raf=None))
    MEKi_index = get_species_index(model, model.monomers.MEKi(mek=None))
    EGF_index = get_species_index(model, model.monomers.EGF(rtk=None))
    initials_pre = df_eq.iloc[-1, :len(model.species)].copy()
    initials_pre[RAFi_index] = rafi
    initials_pre[MEKi_index] = meki
    initials_pre[EGF_index] = 0.0
    tspan_pretrt = np.linspace(0, pretrt, N_time_points)  
    df_pre = sim.run(tspan=tspan_pretrt, initials=initials_pre).dataframe
    df_pre['time'] = df_pre.index
    df_pre['time'] = df_pre['time']-pretrt
    df_pre['time'].iloc[-1] = 0
    df_pre.reset_index(drop=True, inplace=True)
    df_pre.set_index('time', inplace=True)
    
    # run a time-course simulation for the EGF perturbation phase
    tspan_trt = np.linspace(0, trt, N_time_points)
    initials_trt = df_pre.iloc[-1, :len(model.species)].copy()
    initials_trt[RAFi_index] = rafi
    initials_trt[MEKi_index] = meki
    initials_trt[EGF_index] = egfc / model.expressions['m_Da_EGF'].get_value()
    df_trt = sim.run(tspan=tspan_trt, initials=initials_trt).dataframe
    
    # concatenate pretreatment and EGFR perturbations
    obs = pd.concat([df_pre, df_trt.iloc[1:]])[df_pre.keys()[len(model.species):]]
    # clamp small fluctuations due to integrator precision issues
    obs.loc[:, (obs < 1e-10).all()] = 0
    # add columns containing simulation parameters
    settings = {
        'Cell_line': 'A375_sim',
        'Parameter_set': param,
        'Drug A': 'Vemurafenib',
        'Drug B': 'Cobimetinib',
        'Concentration A (uM)': rafi,
        'Concentration B (uM)': meki,
        'Time A (h)': pretrt,
        'Time B (h)': pretrt,
        'EGF (ng/mL)': egfc,
        'EGF total duration (h)': trt,
    }
    for column, value in settings.items():
         obs[column] = value
    # write simulation results to file 
    if i == 0: 
       obs.to_csv(out_filename, mode='w', header=True)
    else:
       obs.to_csv(out_filename, mode='a', header=False)
        
    # update param set used
    param_prev = param

Compiling /home/jovyan/.cache/cython/inline/_cython_inline_bca377cf0e656f6b1afd434c991ead55.pyx because it changed.
[1/1] Cythonizing /home/jovyan/.cache/cython/inline/_cython_inline_bca377cf0e656f6b1afd434c991ead55.pyx

